In [1]:
import os, re, math
import rasterio
from pyproj import Transformer

# ====== EDIT THESE ======
lulc_file      = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"  # your LULC raster
dem_tiles_dir  = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets"       # folder with all .tif/.hgt tiles (NRSC + SRTM)
merged_dem_file = r"C:\Users\Ankit\Downloads\merged_DEM.tif"           # optional: your merged DEM (used only for reporting)
# ========================

def to_wgs84_bounds(path):
    """Return (minlon, minlat, maxlon, maxlat) for a raster, reprojected to EPSG:4326."""
    with rasterio.open(path) as src:
        minx, miny, maxx, maxy = src.bounds
        src_crs = src.crs
    if src_crs is None:
        raise ValueError(f"{path} has no CRS.")
    if str(src_crs).upper() in ("EPSG:4326", "WGS84", "EPSG:4326+WGS84"):
        return (minx, miny, maxx, maxy)
    tr = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
    ll = tr.transform(minx, miny)
    ur = tr.transform(maxx, maxy)
    # ensure proper ordering
    minlon, minlat = min(ll[0], ur[0]), min(ll[1], ur[1])
    maxlon, maxlat = max(ll[0], ur[0]), max(ll[1], ur[1])
    return (minlon, minlat, maxlon, maxlat)

def code_from_latlon(lat, lon):
    ns = "N" if lat >= 0 else "S"
    ew = "E" if lon >= 0 else "W"
    return f"{ns}{abs(lat):02d}{ew}{abs(lon):03d}"

def tiles_from_bounds(minlon, minlat, maxlon, maxlat):
    """
    Return the set of 1°×1° tile codes (e.g., N31E080) covering the bbox in EPSG:4326.
    Handles exact-integer boundaries so we don't over-request tiles.
    """
    # If the max boundary sits exactly on an integer, step it back by a tiny epsilon
    # so we don't include the next tile that starts at that integer.
    def adjust_max(v):
        if abs(v - round(v)) < 1e-12:
            # nudge slightly downward
            return math.nextafter(v, -float('inf'))
        return v

    maxlon = adjust_max(maxlon)
    maxlat = adjust_max(maxlat)

    min_lat = math.floor(minlat)
    max_lat = math.floor(maxlat)
    min_lon = math.floor(minlon)
    max_lon = math.floor(maxlon)

    out = set()
    for la in range(min_lat, max_lat + 1):
        for lo in range(min_lon, max_lon + 1):
            out.add(code_from_latlon(la, lo))
    return out

# 1) Required tiles for the LULC extent (in EPSG:4326)
lulc_minlon, lulc_minlat, lulc_maxlon, lulc_maxlat = to_wgs84_bounds(lulc_file)
required_tiles = tiles_from_bounds(lulc_minlon, lulc_minlat, lulc_maxlon, lulc_maxlat)

# 2) Tiles actually present in your folder (NRSC + SRTM styles)
#    This regex allows ANY characters between N## and E###
name_re = re.compile(r'([NS]\d{2}).*?([EW]\d{3})', re.IGNORECASE)
available_tiles = set()

for fn in os.listdir(dem_tiles_dir):
    if not fn.lower().endswith((".tif", ".tiff", ".hgt", ".img", ".bil", ".zip", ".gz")):
        continue
    m = name_re.search(fn)
    if m:
        latcode = m.group(1).upper()  # e.g., N31
        loncode = m.group(2).upper()  # e.g., E080
        # normalize (keep leading zeros; pattern guarantees 2 + 3 digits)
        available_tiles.add(f"{latcode}{loncode}")

# 3) (Optional) Tiles spanned by your merged DEM's extent (for your info only)
merged_tiles = set()
if os.path.exists(merged_dem_file):
    dminlon, dminlat, dmaxlon, dmaxlat = to_wgs84_bounds(merged_dem_file)
    merged_tiles = tiles_from_bounds(dminlon, dminlat, dmaxlon, dmaxlat)

# 4) Report
print("LULC extent (EPSG:4326):")
print(f"  Lon: {lulc_minlon:.6f} .. {lulc_maxlon:.6f}")
print(f"  Lat: {lulc_minlat:.6f} .. {lulc_maxlat:.6f}\n")

print(f"Required tiles for LULC: {sorted(required_tiles)}\n")
print(f"Tiles found in folder  : {sorted(available_tiles)}\n")

missing = sorted(required_tiles - available_tiles)
if missing:
    print("⚠ Missing tiles to download:")
    for t in missing:
        print(" ", t)
else:
    print("✅ No tiles missing based on filenames — your folder covers the LULC bbox.")

# Extra: sanity-check using merged DEM extent (informational)
if merged_tiles:
    not_in_merged = sorted(required_tiles - merged_tiles)
    if not_in_merged:
        print("\nℹ LULC tiles not spanned by the merged DEM's extent (check your merge or CRS):")
        for t in not_in_merged:
            print(" ", t)
    else:
        print("\n✅ Merged DEM’s geographic extent spans all required tiles.")


LULC extent (EPSG:4326):
  Lon: 77.002176 .. 80.998269
  Lat: 28.442303 .. 31.562595

Required tiles for LULC: ['N28E077', 'N28E078', 'N28E079', 'N28E080', 'N29E077', 'N29E078', 'N29E079', 'N29E080', 'N30E077', 'N30E078', 'N30E079', 'N30E080', 'N31E077', 'N31E078', 'N31E079', 'N31E080']

Tiles found in folder  : ['N28E077', 'N28E078', 'N28E079', 'N28E080', 'N29E077', 'N29E078', 'N29E079', 'N29E080', 'N30E077', 'N30E078', 'N30E079', 'N30E080', 'N31E077', 'N31E078', 'N31E079', 'N31E080']

✅ No tiles missing based on filenames — your folder covers the LULC bbox.

✅ Merged DEM’s geographic extent spans all required tiles.
